In [1]:
from pyspark.sql import SparkSession
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.sql.functions import col, log

import mlflow
import mlflow.spark

# %%
spark = SparkSession.builder \
    .appName("SECOP_MLflow") \
    .master("local[*]") \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
26/02/14 06:46:41 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


### **Configurar MLflow tracking server y experimento**

In [2]:
mlflow.set_tracking_uri("http://mlflow:5000")

In [3]:
experiment_name="secop_prediccion"
mlflow.set_experiment(experiment_name)

2026/02/14 06:46:43 INFO mlflow.tracking.fluent: Experiment with name 'secop_prediccion' does not exist. Creating a new experiment.


<Experiment: artifact_location='file:///opt/mlflow/mlruns/968059932841652694', creation_time=1771051603678, experiment_id='968059932841652694', last_update_time=1771051603678, lifecycle_stage='active', name='secop_prediccion', tags={}>

In [4]:
df = spark.read.parquet("/opt/spark-data/raw/secop_ml_ready.parquet")
df = df.withColumnRenamed("valor_del_contrato_log", "label") \
       .withColumnRenamed("features_pca", "features") \
       .filter(col("label").isNotNull())

train, test = df.randomSplit([0.8, 0.2], seed=42)

print(f"Train: {train.count():,}")
print(f"Test: {test.count():,}")

# %%
evaluator_rmse = RegressionEvaluator(
    labelCol="label",
    predictionCol="prediction",
    metricName="rmse"
)

evaluator_mae = RegressionEvaluator(
    labelCol="label",
    predictionCol="prediction",
    metricName="mae"
)
evaluator_r2 = RegressionEvaluator(
    labelCol="label",
    predictionCol="prediction",
    metricName="r2"
)

Train: 80,153
Test: 19,847


### **Registrar experimento baseline con log_param/log_metric**

In [5]:
print("Experimento 1")
with mlflow.start_run(run_name="baseline_model"):
    # definamos parametros
    reg_param = 0.1
    elastic_param = 0.0
    max_iter = 100

    # Log de Parámetros
    mlflow.log_param("regParam", reg_param)
    mlflow.log_param("elasticParam", elastic_param)
    mlflow.log_param("maxIaram", max_iter)

    # Entrenar Modelo
    lr= LinearRegression(
        featuresCol="features",
        labelCol="label",
        regParam=reg_param,
        elasticNetParam=elastic_param,
        maxIter=max_iter
    )

    model = lr.fit(train)

    # Generar Predicciones
    predictions = model.transform(test)
    
    # Evaluar modelo
    rmse = evaluator_rmse.evaluate(predictions)
    mae = evaluator_mae.evaluate(predictions)
    r2 = evaluator_r2.evaluate(predictions)

    # Log de metricas
    mlflow.log_metric("rmse", rmse)
    mlflow.log_metric("mae", mae)
    mlflow.log_metric("r2", r2)

    # Guardar modelo
    mlflow.spark.log_model(model, "model")

    

    print(f"✓ RMSE: ${rmse:,.2f}")
    print(f"✓ MAE: ${mae:,.2f}")
    print(f"✓ R²: {r2:.4f}")




2026/02/14 06:46:57 WARNING mlflow.utils.git_utils: Failed to import Git (the Git executable is probably not on your PATH), so Git SHA is not available. Error: Failed to initialize: Bad git executable.
The git executable must be specified in one of the following ways:
    - be included in your $PATH
    - be set via $GIT_PYTHON_GIT_EXECUTABLE
    - explicitly set via git.refresh(<full-path-to-git-executable>)

All git commands will error until this is rectified.

This initial message can be silenced or aggravated in the future by setting the
$GIT_PYTHON_REFRESH environment variable. Use one of the following values:
    - quiet|q|silence|s|silent|none|n|0: for no message or exception
    - warn|w|warning|log|l|1: for a warning message (logging level CRITICAL, displayed by default)
    - error|e|exception|raise|r|2: for a raised exception

Example:
    export GIT_PYTHON_REFRESH=quiet



Experimento 1


26/02/14 06:46:58 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors
26/02/14 06:46:59 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
26/02/14 06:46:59 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.VectorBLAS
26/02/14 06:47:00 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.lapack.JNILAPACK
/usr/local/lib/python3.11/site-packages/_distutils_hack/__init__.py:30: UserWarning: Setuptools is replacing distutils. Support for replacing an already imported distutils is deprecated. In the future, this condition will fail. Register concerns at https://github.com/pypa/setuptools/issues/new?template=distutils-deprecation.yml
  warnings.warn(


✓ RMSE: $0.89
✓ MAE: $0.46
✓ R²: 0.7385


### **Registrar multiples modelos (Ridge, Lasso, ElasticNet)**

In [6]:
print("REGISTRANDO MODELOS CON DIFERENTE REGULARIZACIÓN")

# Lista de configuraciones
experiments = [
    {"name": "ridge_l2_regression", "reg": 0.1, "elastic": 0.0, "type": "Ridge"},
    {"name": "lasso_l1_regression", "reg": 0.1, "elastic": 1.0, "type": "Lasso"},
    {"name": "elasticnet_l1_l2", "reg": 0.1, "elastic": 0.5, "type": "ElasticNet"},
]

# Evaluadores
evaluator_rmse = RegressionEvaluator(labelCol="label", predictionCol="prediction", metricName="rmse")
evaluator_mae = RegressionEvaluator(labelCol="label", predictionCol="prediction", metricName="mae")
evaluator_r2 = RegressionEvaluator(labelCol="label", predictionCol="prediction", metricName="r2")

for exp in experiments:

    print(f"\n=== EXPERIMENTO: {exp['type']} ===")

    with mlflow.start_run(run_name=exp["name"]):

        # -----------------------------
        # Log de parámetros
        # -----------------------------
        mlflow.log_param("regParam", exp["reg"])
        mlflow.log_param("elasticNetParam", exp["elastic"])
        mlflow.log_param("maxIter", 100)
        mlflow.log_param("model_type", exp["type"])

        # -----------------------------
        # Entrenar modelo
        # -----------------------------
        lr = LinearRegression(
            featuresCol="features",
            labelCol="label",
            regParam=exp["reg"],
            elasticNetParam=exp["elastic"],
            maxIter=100
        )

        model = lr.fit(train)

        # -----------------------------
        # Predicciones
        # -----------------------------
        predictions = model.transform(test)

        # -----------------------------
        # Métricas
        # -----------------------------
        rmse = evaluator_rmse.evaluate(predictions)
        mae = evaluator_mae.evaluate(predictions)
        r2 = evaluator_r2.evaluate(predictions)

        mlflow.log_metric("rmse", rmse)
        mlflow.log_metric("mae", mae)
        mlflow.log_metric("r2", r2)

        # -----------------------------
        # Guardar modelo
        # -----------------------------
        mlflow.spark.log_model(model, "model")

        print(f"✓ RMSE: ${rmse:,.2f}")
        print(f"✓ MAE:  ${mae:,.2f}")
        print(f"✓ R²:   {r2:.4f}")

print("\n" + "="*60)
print("✓ 3 MODELOS REGISTRADOS EN MLFLOW")
print("✓ Accede a MLflow UI: http://localhost:5000")
print("="*60)


REGISTRANDO MODELOS CON DIFERENTE REGULARIZACIÓN

=== EXPERIMENTO: Ridge ===
✓ RMSE: $0.89
✓ MAE:  $0.46
✓ R²:   0.7385

=== EXPERIMENTO: Lasso ===
✓ RMSE: $0.92
✓ MAE:  $0.50
✓ R²:   0.7209

=== EXPERIMENTO: ElasticNet ===
✓ RMSE: $0.91
✓ MAE:  $0.48
✓ R²:   0.7312

✓ 3 MODELOS REGISTRADOS EN MLFLOW
✓ Accede a MLflow UI: http://localhost:5000


Registrar múltiples métricas permite evaluar el modelo desde diferentes perspectivas. RMSE penaliza fuertemente errores grandes, MAE mide el error promedio absoluto sin amplificar outliers y R² indica qué proporción de la varianza es explicada por el modelo. Un modelo puede tener buen RMSE pero bajo R², o viceversa. Registrar varias métricas permite tomar decisiones más informadas y comparar modelos con mayor profundidad.

### **Explorar y comparar runs en MLflow UI**

**¿Qué modelo tiene el menor RMSE?**  
El modelo Ridge (L2) presenta el menor RMSE dentro de los experimentos evaluados, aunque la diferencia frente a Lasso, ElasticNet y el modelo baseline es mínima y no representa una mejora significativa en términos prácticos.

**¿Hay correlación entre regularización y rendimiento?**  
No se observa una correlación clara entre el nivel o tipo de regularización y el rendimiento del modelo. Los valores de RMSE, MAE y R² son prácticamente iguales en los cuatro modelos, lo que indica que la regularización aplicada (regParam=0.1) no genera un impacto relevante en la capacidad predictiva bajo este conjunto de datos.

**¿Cómo podrías compartir estos resultados con tu equipo?**  
Los resultados pueden compartirse mediante la interfaz de MLflow UI, utilizando la opción de comparación de runs o el botón “Share” del experimento. También es posible exportar métricas, descargar artefactos registrados (modelos y reportes) o integrar MLflow con un repositorio compartido para que el equipo pueda reproducir y auditar los experimentos.

### **Agregar artefactos personalizados (reportes, graficos)**

In [14]:
import matplotlib.pyplot as plt
import pandas as pd
import tempfile

print("\n===  Modelo con Artefactos ===")

with mlflow.start_run(run_name="model_with_artifacts"):

    # Usamos los mejores hiperparámetros encontrados
    reg_param = 0.1
    elastic_param = 0.0  # Ridge
    max_iter = 100

    mlflow.log_param("regParam", reg_param)
    mlflow.log_param("elasticNetParam", elastic_param)
    mlflow.log_param("maxIter", max_iter)
    mlflow.log_param("model_type", "Ridge")

    # Entrenar modelo
    lr = LinearRegression(
        featuresCol="features",
        labelCol="label",
        regParam=reg_param,
        elasticNetParam=elastic_param,
        maxIter=max_iter
    )

    model = lr.fit(train)
    predictions = model.transform(test)

    # Evaluar métricas
    rmse = evaluator_rmse.evaluate(predictions)
    mae = evaluator_mae.evaluate(predictions)
    r2 = evaluator_r2.evaluate(predictions)

    mlflow.log_metric("rmse", rmse)
    mlflow.log_metric("mae", mae)
    mlflow.log_metric("r2", r2)

    # ==============================
    # 1️⃣ REPORTE EN TEXTO
    # ==============================

    report = f"""
    REPORTE DE MODELO
    ==================
    Modelo: Ridge Regression
    regParam: {reg_param}
    elasticNetParam: {elastic_param}
    maxIter: {max_iter}

    MÉTRICAS:
    RMSE: ${rmse:,.2f}
    MAE: ${mae:,.2f}
    R²: {r2:.4f}
    """

    mlflow.log_text(report, "model_report.txt")

    # ==============================
    # 2️⃣ GRÁFICO REAL VS PREDICHO
    # ==============================

    # Convertir pequeña muestra a pandas
    sample_pd = predictions.select("label", "prediction").limit(1000).toPandas()

    plt.figure(figsize=(6,6))
    plt.scatter(sample_pd["label"], sample_pd["prediction"], alpha=0.5)
    plt.xlabel("Valor Real")
    plt.ylabel("Valor Predicho")
    plt.title("Predicciones vs Valores Reales")

    # Línea ideal
    min_val = sample_pd["label"].min()
    max_val = sample_pd["label"].max()
    plt.plot([min_val, max_val], [min_val, max_val], 'r--')

    temp_plot_path = tempfile.mktemp(suffix=".png")
    plt.savefig(temp_plot_path)
    plt.close()

    mlflow.log_artifact(temp_plot_path)

    # ==============================
    # 3️⃣ Guardar modelo
    # ==============================

    mlflow.spark.log_model(model, "model")

    print(f"✓ RMSE: ${rmse:,.2f}")
    print("✓ Reporte y gráfico guardados en MLflow")



===  Modelo con Artefactos ===
✓ RMSE: $0.89
✓ Reporte y gráfico guardados en MLflow


**Preguntas de Reflexión**

**1. ¿Qué ventajas tiene MLflow sobre guardar métricas en archivos CSV?**  
MLflow centraliza parámetros, métricas, modelos y artefactos en un mismo sistema versionado, permitiendo comparar experimentos fácilmente. Un CSV no mantiene trazabilidad, control de versiones ni almacenamiento de modelos asociados.

**2. ¿Cómo implementarías MLflow en un proyecto de equipo?**  
Se configuraría un tracking server centralizado accesible por todos los miembros. Cada experimento se registraría automáticamente desde los notebooks o pipelines, permitiendo auditoría, comparación y control de versiones colaborativo.

**3. ¿Qué artefactos adicionales guardarías además del modelo?**  
Gráficos de métricas, matrices de error, reportes de validación, pipelines completos de preprocesamiento, logs de entrenamiento, configuración del entorno y snapshots del dataset utilizado.

**4. ¿Cómo automatizarías el registro de experimentos?**  
Integrando MLflow dentro de pipelines CI/CD o jobs programados, donde cada entrenamiento registre automáticamente parámetros, métricas y artefactos sin intervención manual.


## **Notebook_11**

### **Configurar MLflow y MlflowClient**

In [20]:
# Elimina el modelo en el dado caso de que se cargue de mas 

# from mlflow.tracking import MlflowClient

# client = MlflowClient()
# model_name = "secop_prediccion_contratos"

# Elimina el modelo completo del Model Registry
# client.delete_registered_model(model_name)
# print(f"Modelo '{model_name}' eliminado del Registry.")


Modelo 'secop_prediccion_contratos' eliminado del Registry.


In [21]:
from mlflow.tracking import MlflowClient

# Cliente del Model Registry
client = MlflowClient()

# Nombre del modelo en el Registry
model_name = "secop_prediccion_contratos"

print(f"MLflow URI: {mlflow.get_tracking_uri()}")
print(f"Modelo en Registry: {model_name}")


MLflow URI: http://mlflow:5000
Modelo en Registry: secop_prediccion_contratos


### **Entrenar y registrar modelo v1 (baseline)**

In [22]:
mlflow.set_experiment("SECOP_Model_Registry")

with mlflow.start_run(run_name="model_v1_baseline") as run:

    # Modelo SIN regularización
    lr = LinearRegression(
        featuresCol="features",
        labelCol="label",
        regParam=0.0,
        elasticNetParam=0.0,
        maxIter=100
    )

    model_v1 = lr.fit(train)

    predictions_v1 = model_v1.transform(test)
    rmse_v1 = evaluator_rmse.evaluate(predictions_v1)

    # Log
    mlflow.log_param("version", "1.0")
    mlflow.log_param("model_type", "baseline")
    mlflow.log_param("regParam", 0.0)
    mlflow.log_metric("rmse", rmse_v1)

    # Registro en el Model Registry
    mlflow.spark.log_model(
        spark_model=model_v1,
        artifact_path="model",
        registered_model_name=model_name
    )

    run_id_v1 = run.info.run_id
    print(f"Modelo v1 registrado | RMSE: ${rmse_v1:,.2f}")


26/02/14 06:55:19 WARN Instrumentation: [c05bf145] regParam is zero, which might cause numerical instability and overfitting.
Successfully registered model 'secop_prediccion_contratos'.
2026/02/14 06:55:30 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: secop_prediccion_contratos, version 1
Created version '1' of model 'secop_prediccion_contratos'.


Modelo v1 registrado | RMSE: $0.89


### **Entrenar y registrar modelo v2 (mejorado)**

¿Por qué versionar modelos en lugar de sobrescribir?

Porque permite trazabilidad, rollback seguro, auditoría y comparación histórica entre modelos. En producción nunca se debe perder el historial de versiones.

In [23]:
with mlflow.start_run(run_name="model_v2_regularized") as run:

    lr = LinearRegression(
        featuresCol="features",
        labelCol="label",
        regParam=0.1,
        elasticNetParam=0.5,
        maxIter=100
    )

    model_v2 = lr.fit(train)

    predictions_v2 = model_v2.transform(test)
    rmse_v2 = evaluator_rmse.evaluate(predictions_v2)

    mlflow.log_param("version", "2.0")
    mlflow.log_param("model_type", "regularized")
    mlflow.log_param("regParam", 0.1)
    mlflow.log_param("elasticNetParam", 0.5)
    mlflow.log_metric("rmse", rmse_v2)

    mlflow.spark.log_model(
        spark_model=model_v2,
        artifact_path="model",
        registered_model_name=model_name
    )

    print(f"Modelo v2 registrado | RMSE: ${rmse_v2:,.2f}")

Registered model 'secop_prediccion_contratos' already exists. Creating a new version of this model...
2026/02/14 06:55:42 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: secop_prediccion_contratos, version 2


Modelo v2 registrado | RMSE: $0.91


Created version '2' of model 'secop_prediccion_contratos'.


In [25]:
print("\nComparación:")
print(f"v1 RMSE: ${rmse_v1:,.2f}")
print(f"v2 RMSE: ${rmse_v2:,.2f}")
print(f"Mejor modelo: {'v2' if rmse_v2 < rmse_v1 else 'v1'}")



Comparación:
v1 RMSE: $0.89
v2 RMSE: $0.91
Mejor modelo: v1


### **Gestionar stages: None → Staging → Production → Archived**

In [26]:
print("GESTIÓN DE VERSIONES EN MODEL REGISTRY")
print("="*60)

# Listar versiones registradas
model_versions = client.search_model_versions(f"name='{model_name}'")

print(f"\nVersiones actuales del modelo '{model_name}':")
for mv in model_versions:
    print(f"  - Versión {mv.version} | Stage: {mv.current_stage}")

# Determinar mejor versión según RMSE
print("\nComparando métricas:")
print(f"  v1 RMSE: ${rmse_v1:,.2f}")
print(f"  v2 RMSE: ${rmse_v2:,.2f}")

best_version = 2 if rmse_v2 < rmse_v1 else 1
worst_version = 1 if best_version == 2 else 2

print(f"\nMejor versión: v{best_version}")

# Promover mejor versión a Staging
client.transition_model_version_stage(
    name=model_name,
    version=best_version,
    stage="Staging"
)

print(f"v{best_version} -> Staging")

# Simular validación (aquí ya se sabe cuál es mejor)
client.transition_model_version_stage(
    name=model_name,
    version=best_version,
    stage="Production"
)

print(f"v{best_version} -> Production")

# Archivar versión anterior
client.transition_model_version_stage(
    name=model_name,
    version=worst_version,
    stage="Archived"
)

print(f"v{worst_version} -> Archived")

# Verificar estado final
print("\nEstado final de versiones:")
model_versions = client.search_model_versions(f"name='{model_name}'")
for mv in model_versions:
    print(f"  - Versión {mv.version} | Stage: {mv.current_stage}")



GESTIÓN DE VERSIONES EN MODEL REGISTRY

Versiones actuales del modelo 'secop_prediccion_contratos':
  - Versión 2 | Stage: None
  - Versión 1 | Stage: None

Comparando métricas:
  v1 RMSE: $0.89
  v2 RMSE: $0.91

Mejor versión: v1
v1 -> Staging


/tmp/ipykernel_19/2731657312.py:22: FutureWarning: ``mlflow.tracking.client.MlflowClient.transition_model_version_stage`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/2.9.2/model-registry.html#migrating-from-stages
  client.transition_model_version_stage(
/tmp/ipykernel_19/2731657312.py:31: FutureWarning: ``mlflow.tracking.client.MlflowClient.transition_model_version_stage`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/2.9.2/model-registry.html#migrating-from-stages
  client.transition_model_version_stage(
/tmp/ipykernel_19/2731657312.py:40: FutureWarning: ``mlflow.tracking.client.MlflowClient.transition_model_version_stage`` is deprecated since 2.9.0. Model registr

v1 -> Production
v2 -> Archived

Estado final de versiones:
  - Versión 2 | Stage: Archived
  - Versión 1 | Stage: Production


### **Agregar metadata y descripcion al modelo**

**¿Qué información mínima debería tener cada versión?**

Cada versión de modelo debe incluir como mínimo:
- Métrica principal de validación (RMSE u otra relevante)
- Fecha de entrenamiento
- Dataset utilizado
- Tipo de modelo y configuración principal
- Responsable o autor
- Estado del modelo (Staging o Production)

Esto garantiza trazabilidad, auditoría y reproducibilidad en entornos productivos.


In [27]:
# Identificar versión en Production
from mlflow.tracking import MlflowClient

client = MlflowClient()

print("\nBuscando versión en Production...")

production_version = None

for mv in client.search_model_versions(f"name='{model_name}'"):
    if mv.current_stage == "Production":
        production_version = mv.version
        break

print(f"Versión en Production: {production_version}")



Buscando versión en Production...
Versión en Production: 1


In [28]:
# Agregar Descripción Profesional
client.update_model_version(
    name=model_name,
    version=production_version,
    description=f"""
Modelo oficial en producción para predicción de valor de contratos SECOP.

• Versión: {production_version}
• RMSE validación: ${rmse_v1:,.2f}
• Dataset: secop_ml_ready.parquet
• Features: PCA
• Autor: Diego_Gomez_and_Victor_Diaz
• Fecha: 2026-02-13

Modelo seleccionado tras comparación entre baseline y modelo regularizado.
"""
)

print(f"✓ Metadata agregada a versión {production_version}")


✓ Metadata agregada a versión 1


In [29]:
# Agregar Tags 
client.set_model_version_tag(
    name=model_name,
    version=production_version,
    key="status",
    value="validated"
)

client.set_model_version_tag(
    name=model_name,
    version=production_version,
    key="area",
    value="finanzas_publicas"
)

client.set_model_version_tag(
    name=model_name,
    version=production_version,
    key="framework",
    value="SparkML"
)

print("✓ Tags agregados correctamente")


✓ Tags agregados correctamente


### **Cargar modelo desde Registry para prediccion**

In [30]:
print("CARGANDO MODELO DESDE PRODUCTION")
print("="*60)

# Definir URI usando nombre y stage (NO ruta de archivo)
model_uri = f"models:/{model_name}/Production"

# Cargar modelo desde Registry
loaded_model = mlflow.spark.load_model(model_uri)

print(f"Modelo cargado desde: {model_uri}")
print(f"Tipo de objeto: {type(loaded_model)}")

# Verificar que funciona haciendo predicciones
test_predictions = loaded_model.transform(test)

# Evaluar nuevamente
test_rmse = evaluator_rmse.evaluate(test_predictions)

print(f"\nRMSE verificación: ${test_rmse:,.2f}")
print("="*60)


/usr/local/lib/python3.11/site-packages/mlflow/store/artifact/utils/models.py:32: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/2.9.2/model-registry.html#migrating-from-stages
  latest = client.get_latest_versions(name, None if stage is None else [stage])
2026/02/14 06:58:26 INFO mlflow.spark: 'models:/secop_prediccion_contratos/Production' resolved as 'file:///opt/mlflow/mlruns/183258816077697738/afaf868be7984fc39c07b46e91d23864/artifacts/model'


CARGANDO MODELO DESDE PRODUCTION


2026/02/14 06:58:26 INFO mlflow.spark: URI 'models:/secop_prediccion_contratos/Production/sparkml' does not point to the current DFS.
2026/02/14 06:58:26 INFO mlflow.spark: File 'models:/secop_prediccion_contratos/Production/sparkml' not found on DFS. Will attempt to upload the file.


Modelo cargado desde: models:/secop_prediccion_contratos/Production
Tipo de objeto: <class 'pyspark.ml.pipeline.PipelineModel'>

RMSE verificación: $0.89


1. ¿Cómo harías rollback si el modelo en Production falla?

Si el modelo en Production presenta fallos, realizaría rollback promoviendo la versión anterior desde "Archived" o "Staging" nuevamente a "Production" utilizando transition_model_version_stage(). Esto permite revertir el modelo activo sin modificar el código de producción, ya que el sistema siempre carga el modelo por nombre y stage.

2. ¿Qué criterios usarías para promover un modelo de Staging a Production?

Promovería un modelo a Production únicamente si:

  - Presenta mejor RMSE/MAE/R² que la versión actual.
  - Supera validaciones técnicas y de negocio.
  - Es estable en pruebas controladas.
  - No introduce sesgos o comportamientos inesperados.
  - Ha sido revisado y aprobado por el equipo responsable.

3. ¿Cómo implementarías A/B testing con el Model Registry?

Implementaría A/B testing desplegando dos versiones diferentes del modelo (por ejemplo, Production y Staging) y enviando un porcentaje del tráfico a cada uno. Posteriormente compararía métricas en producción (errores reales, impacto financiero, latencia) para determinar cuál modelo ofrece mejor rendimiento antes de hacer la promoción definitiva.

4. ¿Quién debería tener permisos para promover modelos a Production?

Solo deberían tener permisos para promover modelos a Production los roles responsables de MLOps o Data Science Lead, ya que este proceso impacta directamente el entorno productivo. Esto evita errores humanos y mantiene control y trazabilidad sobre cambios críticos.

## **Notebook_12**

1. ¿Por qué cargar desde el Registry en lugar de una ruta de archivo? ¿Qué ventajas tiene para un sistema de producción?

  - Permite cambiar la versión en Production sin modificar código.
  - Facilita rollback inmediato.
  - Centraliza control y gobernanza.
  - Garantiza trazabilidad y versionamiento.

2. ¿Qué pasaría si no hay modelo en Production?

El sistema lanzaría un error porque no existe una versión promovida a ese stage.

3. ¿Cómo manejar ese error?

  - Capturar la excepción (como hicimos arriba).
  - Enviar alerta.
  - Cargar versión anterior estable.
  - Detener el pipeline si es crítico.

### **Cargar modelo en Production desde MLflow Registry**

In [31]:
mlflow.set_tracking_uri("http://mlflow:5000")
# nombre del modelo registrado
model_name = "secop_prediccion_contratos"
model_uri = f"models:/{model_name}/Production"

print("CARGANDO MODELO DESDE REGISTRY")
print("="*60)
print(f"URI: {model_uri}")

try:
    production_model = mlflow.spark.load_model(model_uri)
    print("✓ Modelo cargado correctamente")
    print(f"Tipo: {type(production_model)}")

except Exception as e:
    print("No existe modelo en Production")
    print("Error:", e)


2026/02/14 06:58:52 INFO mlflow.spark: 'models:/secop_prediccion_contratos/Production' resolved as 'file:///opt/mlflow/mlruns/183258816077697738/afaf868be7984fc39c07b46e91d23864/artifacts/model'


CARGANDO MODELO DESDE REGISTRY
URI: models:/secop_prediccion_contratos/Production


2026/02/14 06:58:52 INFO mlflow.spark: URI 'models:/secop_prediccion_contratos/Production/sparkml' does not point to the current DFS.
2026/02/14 06:58:53 INFO mlflow.spark: File 'models:/secop_prediccion_contratos/Production/sparkml' not found on DFS. Will attempt to upload the file.


✓ Modelo cargado correctamente
Tipo: <class 'pyspark.ml.pipeline.PipelineModel'>


### **Preparar datos nuevos para prediccion**

In [32]:
df_new = spark.read.parquet("/opt/spark-data/raw/secop_ml_ready.parquet")
df_new = df_new.withColumnRenamed("features_pca", "features")

# Simular que no tenemos label en producción
df_new_no_label = df_new.drop("valor_del_contrato_log")

print(f"Registros para predicción: {df_new_no_label.count():,}")
print("Columnas disponibles:")
print(df_new_no_label.columns)

Registros para predicción: 100,000
Columnas disponibles:
['features']


### **Generar predicciones batch con timestamp**

In [33]:
from pyspark.sql.functions import current_timestamp

predictions_batch = production_model.transform(df_new_no_label)

predictions_batch = predictions_batch.withColumn(
    "prediction_timestamp",
    current_timestamp()
)

predictions_batch.select(
    "features",
    "prediction",
    "prediction_timestamp"
).show(10, truncate=False)



+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+------------------+--------------------------+
|features                                                                                                                                                                                                                                                                                                                                          

¿Por qué agregar timestamp?

 - Auditoría
 - Trazabilidad
 - Comparación entre lotes
 - Monitoreo temporal

### **Monitorear predicciones (estadisticas, anomalias, rangos)**

¿Cómo detectar data drift?

  - Comparar media y desviación vs entrenamiento
  - Monitorear distribución por rangos
  - Alertas si cambia > X%
  - Validar features entrantes

Si hay drift, toca tener alertas, Evaluar retraining y Revertir versión

In [34]:
from pyspark.sql.functions import min as spark_min, max as spark_max, avg, stddev, count
from pyspark.sql.functions import exp

predictions_batch = predictions_batch.withColumn(
    "prediction_real",
    exp(col("prediction"))
)

stats = predictions_batch.select(
    spark_min("prediction_real").alias("min_pred"),
    spark_max("prediction_real").alias("max_pred"),
    avg("prediction_real").alias("avg_pred"),
    stddev("prediction_real").alias("std_pred"),
    count("*").alias("total")
).collect()[0]


print("=== ESTADÍSTICAS DE PREDICCIONES ===")
print(f"Total: {stats['total']:,}")
print(f"Mínimo: ${stats['min_pred']:,.2f}")
print(f"Máximo: ${stats['max_pred']:,.2f}")
print(f"Promedio: ${stats['avg_pred']:,.2f}")
print(f"Std: ${stats['std_pred']:,.2f}")

from pyspark.sql.functions import when

prediction_ranges = predictions_batch.select(
    count(when(col("prediction_real") < 10000000, True)).alias("< 10M"),
    count(when((col("prediction_real") >= 10000000) & (col("prediction_real") < 100000000), True)).alias("10M-100M"),
    count(when((col("prediction_real") >= 100000000) & (col("prediction_real") < 1000000000), True)).alias("100M-1B"),
    count(when(col("prediction_real") >= 1000000000, True)).alias("> 1B")
)


print("=== DISTRIBUCIÓN POR RANGOS ===")
prediction_ranges.show()


=== ESTADÍSTICAS DE PREDICCIONES ===
Total: 100,000
Mínimo: $0.00
Máximo: $8,892,625,557,609.46
Promedio: $237,343,677.25
Std: $37,033,573,997.19
=== DISTRIBUCIÓN POR RANGOS ===
+-----+--------+-------+----+
|< 10M|10M-100M|100M-1B|> 1B|
+-----+--------+-------+----+
| 6313|   87957|   5265| 465|
+-----+--------+-------+----+



### **Dectectar anomalias**

In [35]:
anomalias = predictions_batch.filter(col("prediction_real") < 0).count()
print(f"Predicciones negativas: {anomalias}")

Predicciones negativas: 0


### **Guardar resultados en Parquet y CSV**

¿Qué formato usarías para cada caso?
 - Dashboard interno: Parquet
 - Reporte para gerencia: CSV
 - Input para otro sistema: Depende del sistema, pero normalmente, parquet si es otro sistema Big Data / Spark / Data Lake. CSV o JSON si es API o sistema transaccional


In [37]:
predictions_output = "/opt/spark-data/raw/predictions_produccion"

# Guardar en Parquet (formato óptimo para analytics)
predictions_batch.write.mode("overwrite") \
    .parquet(predictions_output + "/parquet")

print(f"Parquet guardado en: {predictions_output}/parquet")

# Guardar en CSV (solo columnas necesarias)
predictions_batch.select(
    "prediction_real",
    "prediction_timestamp"
).write.mode("overwrite") \
 .option("header", "true") \
 .csv(predictions_output + "/csv")

print(f"CSV guardado en: {predictions_output}/csv")


Parquet guardado en: /opt/spark-data/raw/predictions_produccion/parquet
CSV guardado en: /opt/spark-data/raw/predictions_produccion/csv


### **Disenar pipeline de produccion automatizado**

1. **Frecuencia**: ¿Cada hora? ¿Cada día? ¿Bajo demanda?
2. **Orquestador**: ¿Airflow? ¿Cron? ¿Spark Streaming?
3. **Monitoreo**: ¿Cómo detectas si el modelo se degrada?
4. **Reentrenamiento**: ¿Cuándo reentrenar el modelo?
5. **Alertas**: ¿Qué condiciones disparan una alerta?

**Frecuencia**: Cada día (batch nocturno). Los contratos públicos no cambian en tiempo real, por lo que una ejecución diaria es suficiente para mantener las predicciones actualizadas sin sobrecargar infraestructura.

**Orquestador**: Apache Airflow. Permite programar tareas, manejar dependencias (ETL → Scoring → Monitoreo), registrar logs, manejar reintentos y enviar alertas automáticas en caso de fallo.

**Monitoreo**:Detectar degradación mediante: Comparación del RMSE histórico vs actual, cambios significativos en promedio y desviación estándar de predicciones. Detección de data drift (cambio en distribución de features), incremento de outliers o valores extremos. Si las métricas se alejan del rango esperado, el modelo puede estar degradándose.

**Reentrenamiento**: Se debe reentrenar cuando:

  -  El RMSE empeore más de un umbral definido (ej. +10%).
  -  Se detecte data drift significativo.
  -  Se acumulen nuevos datos relevantes (mensual o trimestralmente).

**Alertas**: Se mandaran alertas cuando:

  - El pipeline falla.
  - El RMSE supera un umbral crítico.
  - Se detecta data drift fuerte.
  - Aparecen predicciones fuera de rango esperado.
  - Hay cambios anormales en la distribución diaria.

In [ ]:
# ============================================================
# DISEÑO DEL PIPELINE DE PRODUCCIÓN – SECOP
# ============================================================

# 1. Frecuencia:
#    - Ejecución diaria (batch nocturno).
#    - Hora sugerida: 02:00 AM.
#    - Justificación:
#        • Los contratos públicos no requieren scoring en tiempo real.
#        • Permite consolidar datos del día anterior.
#        • Reduce carga sobre infraestructura en horario laboral.


# 2. Orquestador:
#    - Apache Airflow.
#    - DAG propuesto:
#
#        Task 1: Ingesta datos nuevos (Parquet / DB / S3)
#        Task 2: Cargar modelo desde MLflow Registry (Production)
#        Task 3: Generar predicciones batch
#        Task 4: Aplicar exp() para volver de escala log
#        Task 5: Calcular métricas de monitoreo
#        Task 6: Guardar resultados (Parquet + CSV)
#        Task 7: Validar umbrales y disparar alertas
#
#    - Airflow permite:
#        • Reintentos automáticos
#        • Logs centralizados
#        • Control de dependencias
#        • Alertas por email / Slack


# 3. Monitoreo:
#    - Métricas monitoreadas diariamente:
#        • Promedio de predicciones
#        • Desviación estándar
#        • % de valores > 1B
#        • % de valores < 10M
#    - Comparación contra baseline histórico.
#    - Detección de data drift:
#        • Comparar distribución actual vs entrenamiento.
#        • Monitorear cambios en media y varianza de features PCA.
#
#    - Si el RMSE en validaciones periódicas aumenta >10%,
#      se marca posible degradación del modelo.


# 4. Reentrenamiento:
#    - Condiciones para reentrenar:
#        • RMSE aumenta más de 10% respecto al baseline.
#        • Drift estadístico significativo.
#        • Nuevos datos acumulados (ej: mensual).
#
#    - Proceso:
#        • Ejecutar notebook de entrenamiento (Notebook 10).
#        • Registrar nueva versión en Model Registry.
#        • Comparar contra versión en Production.
#        • Promover solo si mejora métricas.
#
#    - Nunca sobrescribir modelo:
#        • Siempre versionar.
#        • Usar stages (None → Staging → Production).


# 5. Alertas:
#    - Se dispara alerta si:
#        • Pipeline falla.
#        • No se generan predicciones.
#        • Predicciones negativas > 0.
#        • Promedio cambia >20% respecto al histórico.
#        • Máximo excede umbral crítico definido.
#
#    - Canales de alerta:
#        • Email automático.
#        • Slack.
#        • Registro en dashboard interno.


# ============================================================
# Resultado esperado:
# Pipeline automatizado, versionado, monitoreado y gobernado.
# ============================================================


### **Simulacion de scoring continuo por lotes**

In [38]:
from pyspark.sql.functions import avg, exp, col

print("SIMULACIÓN DE SCORING CONTINUO POR LOTES")
print("="*60)

# Dividir en 3 lotes simulados
batches = df_new_no_label.randomSplit([0.33, 0.33, 0.34], seed=42)

for i, batch in enumerate(batches):
    
    # Generar predicciones en escala log
    preds = production_model.transform(batch)
    
    # Convertir a escala real (porque entrenamos en log1p)
    preds = preds.withColumn(
        "prediction_real",
        exp(col("prediction"))
    )
    
    # Calcular estadísticas básicas
    avg_pred = preds.select(avg("prediction_real")).collect()[0][0]
    count_pred = preds.count()
    
    print(f"Lote {i+1}: {count_pred:,} registros | "
          f"Predicción promedio: ${avg_pred:,.2f}")

print("="*60)


SIMULACIÓN DE SCORING CONTINUO POR LOTES
Lote 1: 33,060 registros | Predicción promedio: $49,762,825.50
Lote 2: 33,198 registros | Predicción promedio: $52,075,359.08
Lote 3: 33,742 registros | Predicción promedio: $603,414,466.94


In [39]:
spark.stop()